## Primeiramente, importaremos as bibliotecas necessárias

In [1]:
import warnings

from causalnex.structure import StructureModel
from causalnex.structure.notears import from_pandas
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from causalnex.network import BayesianNetwork
from causalnex.discretiser import Discretiser
from causalnex.evaluation import classification_report
from causalnex.evaluation import roc_auc
from causalnex.inference import InferenceEngine

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import networkx as nx


warnings.filterwarnings("ignore")  # silence warnings

/home/ricardo/.pyenv/versions/3.8.12/envs/causalnex/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Construindo um StructureModel a partir do nosso conhecimento de negócio

In [ ]:
# Instanciando o Modelo
sm = StructureModel()

# Estamos dizendo que saúde é causa para faltas e notas do
# primeiro semestre

sm.add_edges_from([
    ('health', 'absences'),
    ('health', 'G1')
])


In [ ]:
# Visualizando a estrutura
sm.edges

In [ ]:
# Gerando o plot de nossa estrutura

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("supporting_files/01_simple_plot.html")

### Construindo um StructureModel para aprender a estrutura dos nossos dados (NOTEARS)

Ao aprender a estrutura, podemos usar todo o conjunto de dados. Como a estrutura deve ser considerada como um esforço conjunto entre aprendizado de máquina e especialistas no domínio, nem sempre é necessário usar uma divisão de treinamento/teste.

Mas antes de começar, temos que pré-processar os dados para que o algoritmo NOTEARS possa ser usado.

In [2]:
# Fazendo leitura dos nossos dados

data = pd.read_csv('./student-por.csv', delimiter=';')
data.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [3]:
# Dropando colunas com informações sensíveis

drop_col = ['school','sex','age','Mjob', 'Fjob','reason','guardian']
data = data.drop(columns=drop_col)
data.head(5)

,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,U,GT3,A,4,4,2,2,0,yes,no,...,4,3,4,1,1,3,4,0,11,11
1,U,GT3,T,1,1,1,2,0,no,yes,...,5,3,3,1,1,3,2,9,11,11
2,U,LE3,T,1,1,1,2,0,yes,no,...,4,3,2,2,3,3,6,12,13,12
3,U,GT3,T,4,2,1,3,0,no,yes,...,3,2,2,1,1,5,0,14,14,14
4,U,GT3,T,3,3,1,2,0,no,yes,...,4,3,2,1,2,5,0,11,13,13


In [4]:
# Criando uma lista com as colunas não numericas
# para transformarmos em númericas, visto que NOTEARS não
# lida com dados categóricos

struct_data = data.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

['address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']


In [5]:
# Aplicando algortimo de LabelEncoder, para transformar nossas
# colunas categoricas em numéricas

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.head(5)

,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1,0,0,4,4,2,2,0,1,0,...,4,3,4,1,1,3,4,0,11,11
1,1,0,1,1,1,1,2,0,0,1,...,5,3,3,1,1,3,2,9,11,11
2,1,1,1,1,1,1,2,0,1,0,...,4,3,2,2,3,3,6,12,13,12
3,1,0,1,4,2,1,3,0,0,1,...,3,2,2,1,1,5,0,14,14,14
4,1,0,1,3,3,1,2,0,0,1,...,4,3,2,1,2,5,0,11,13,13


In [6]:
# Aplicando o algoritmo NOTEARS em nossos dados

sm = StructureModel()
sm = from_pandas(struct_data)

In [9]:
# Visualizando nossa estrutura

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("supporting_files/01_fully_connected.html")

supporting_files/01_fully_connected.html


A razão pela qual temos um gráfico totalmente conectado aqui é que não aplicamos limiarização às arestas mais fracas. O limite pode ser aplicado especificando o valor para o parâmetro w_threshold em from_pandas ou podemos remover as arestas chamando a função de modelo de estrutura, remove_edges_below_threshold.

In [10]:
sm.remove_edges_below_threshold(0.8)

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.show("supporting_files/01_thresholded.html")

supporting_files/01_thresholded.html


In [11]:
# Após analisar nossa estrutura percebemos que uma das relações não fazem tanto sentido
# para evitar esse erro, iremos rerodar nosso algoritmo NOTEARS com algumas
# restrições adicionais

sm = from_pandas(struct_data, tabu_edges=[("higher", "Medu")], w_threshold=0.8)

In [12]:
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("supporting_files/01_edge_added.html")

supporting_files/01_edge_added.html


In [13]:
# Uma outra maneira de corrigir relações erroneas é trazermos nosso conhecimento de negócio
# e modificar nossa estrutura, seja adicionando ou removendo relações

sm.add_edge("failures", "G1")
sm.remove_edge("Pstatus", "G1")
sm.remove_edge("address", "G1")

In [14]:
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("supporting_files/01_modified_structure.html")

supporting_files/01_modified_structure.html


In [15]:
# Podemos observar que o algoritmo NOTEARS encontrou dois graficos
# de relações separadas. Podemos selecionar somente o maior deles

sm = sm.get_largest_subgraph()
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("supporting_files/01_largest_subgraph.html")

supporting_files/01_largest_subgraph.html


In [17]:
# Após a construção do nosso Modelo Estrutural utilizando NOTEARS
# podemos exporta-lo

nx.drawing.nx_pydot.write_dot(sm, 'graph.dot')

## Fitando uma Distribuição Condicional com Redes Bayesianas

In [18]:
# A partir do nosso modelo estrutural, instanciamos nossa BN
bn = BayesianNetwork(sm)

#### Preparando os Dados Discretizados

As redes bayesianas no CausalNex suportam apenas distribuições discretas. Qualquer distribuição contínuas, ou distribuição com um grande número de categorias, devem ser discretizadas antes de fitarmos uma Rede Bayesiana. Estruturas contendo variáveis ​​com muitos valores possíveis normalmente serão mal ajustados e exibirão baixo desempenho.

Por exemplo, considere P(G2 | G1), onde G1 e G2 têm valores possíveis de 0 a 20. A distribuição de probabilidade condicional discreta é, portanto, especificada usando 21x21 (441) combinações possíveis - a maioria das quais dificilmente observaremos.

O CausalNex fornece alguns métodos auxiliares para facilitar a discretização. Vamos começar reduzindo o número de categorias em algumas das colunas categóricas combinando valores semelhantes. Tornaremos as colunas numéricas em categóricas por discretização e, em seguida, daremos as discretizações rótulos significativos.

##### Discretizando categóricos

In [19]:
# Primeiro iremos reduzir a cardinalidade dos nossos dados categoricos
# definindo dicionarios {'valorantigo': 'valornovo'}.

discretised_data = data.copy()

data_vals = {col: data[col].unique() for col in data.columns}

# Transformando qtde de reprovações para se 0 no-failure
# se não ele have-failure.

failures_map = {v: 'no-failure' if v == [0]
                else 'have-failure' for v in data_vals['failures']}

# Transformando tempo de estudo entre 1 e 2h para 'short-studytime'
# e mais do que 2h para long-studytime.

studytime_map = {v: 'short-studytime' if v in [1,2]
                 else 'long-studytime' for v in data_vals['studytime']}

In [20]:
# Depois de definir nossas condições no dicionários aplicamos
# as alterações atraves do metódo map

discretised_data["failures"] = discretised_data["failures"].map(failures_map)
discretised_data["studytime"] = discretised_data["studytime"].map(studytime_map)

##### Discretizando numéricos

In [21]:
# Antes de discretizarmos os numericos, precisamos transforma-los
# em categóricos. E para isso iremos utilizar a Classe Discretizer
# que nos 'da' faixas de valores. Ex: as ausências serão discretizadas 
# nos intervalos < 1, 1 a 9 e >=10. Cada faixa será rotulada como um número 
# inteiro a partir de zero.

discretised_data["absences"] = Discretiser(method="fixed",
                          numeric_split_points=[1, 10]).transform(discretised_data["absences"].values)

discretised_data["G1"] = Discretiser(method="fixed",
                          numeric_split_points=[10]).transform(discretised_data["G1"].values)

discretised_data["G2"] = Discretiser(method="fixed",
                          numeric_split_points=[10]).transform(discretised_data["G2"].values)

discretised_data["G3"] = Discretiser(method="fixed",
                          numeric_split_points=[10]).transform(discretised_data["G3"].values)

In [22]:
# Para deixar nossa Discretização mais interpretável, iremos aplicar
# o mesmo conceito que utilizamoss nas categóricas

absences_map = {0: "No-absence", 1: "Low-absence", 2: "High-absence"}

G1_map = {0: "Fail", 1: "Pass"}
G2_map = {0: "Fail", 1: "Pass"}
G3_map = {0: "Fail", 1: "Pass"}

discretised_data["absences"] = discretised_data["absences"].map(absences_map)
discretised_data["G1"] = discretised_data["G1"].map(G1_map)
discretised_data["G2"] = discretised_data["G2"].map(G2_map)
discretised_data["G3"] = discretised_data["G3"].map(G3_map)

#### Dividindo nossa base em treino e teste

In [23]:
train, test = train_test_split(discretised_data, train_size=0.9, test_size=0.1, random_state=7)

#### Probabilidade do modelo

Com o modelo de estrutura aprendido anteriormente e os dados discretizados, podemos agora ajustar a distribuição de probabilidade da Rede Bayesiana. 

A primeira etapa é especificar todos os estados que cada nó pode assumir. Isso pode ser feito a partir de dados ou fornecendo um dicionário de valores de nó. 

Usamos o conjunto de dados completo aqui para evitar casos em que os estados em nosso conjunto de teste não existam no conjunto de treinamento. Para aplicações do mundo real, esses estados podem precisar ser fornecidos usando o método do dicionário.

In [24]:
bn = bn.fit_node_states(discretised_data)

#### Ajustando as Distribuições de Probabilidade Condicionais

O método fit_cpds da BayesianNetwork aceita um conjunto de dados para aprender as distribuições de probabilidade condicional (CPDs) de cada nó (relação), juntamente com um método de como fazer esse ajuste.

In [25]:
bn = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")

Assim que tivermos as CPDs, podemos inspecioná-las por meio do método cpds, que é um dicionário de node->cpd.

In [26]:
bn.cpds["G1"]

failures    have-failure                                                 \
higher                no                                                  
schoolsup             no                            yes                   
studytime long-studytime short-studytime long-studytime short-studytime   
G1                                                                        
Fail                0.75        0.806452            0.5            0.75   
Pass                0.25        0.193548            0.5            0.25   

failures                                                                 \
higher               yes                                                  
schoolsup             no                            yes                   
studytime long-studytime short-studytime long-studytime short-studytime   
G1                                                                        
Fail                 0.5        0.612245            0.5            0.75   
Pass                 0.5        0.387755            0.5            0.25   

failures      no-failure                                                 \
higher                no                                                  
schoolsup             no                            yes                   
studytime long-studytime short-studytime long-studytime short-studytime   
G1                                                                        
Fail                 0.5        0.612903            0.5             0.5   
Pass                 0.5        0.387097            0.5             0.5   

failures                                                                 
higher               yes                                                 
schoolsup             no                            yes                  
studytime long-studytime short-studytime long-studytime short-studytime  
G1                                                                       
Fail            0.032967         0.15016       0.111111        0.255814  
Pass            0.967033         0.84984       0.888889        0.744186

#### Prever o estado dado os dados de entrada

O método de previsão da BayesianNetwork nos permite fazer previsões com base nos dados usando a Rede Bayesiana aprendida. Por exemplo, queremos prever se um aluno é reprovado ou aprovado no exame com base nos dados de entrada. Imagine que temos os dados de entrada de um aluno que se parecem com isto:

In [27]:
discretised_data.loc[18, discretised_data.columns != 'G1']


address                     U
famsize                   GT3
Pstatus                     T
Medu                        3
Fedu                        2
traveltime                  1
studytime     short-studytime
failures         have-failure
schoolsup                  no
famsup                    yes
paid                      yes
activities                yes
nursery                   yes
higher                    yes
internet                  yes
romantic                   no
famrel                      5
freetime                    5
goout                       5
Dalc                        2
Walc                        4
health                      5
absences          Low-absence
G2                       Fail
G3                       Fail
Name: 18, dtype: object

Com base nesses dados, queremos prever se esse aluno será reprovado no exame. Intuitivamente, esperaríamos que esse aluno fosse reprovado porque passa menos tempo estudando e já foi reprovado no passado. Vamos ver como nossa Rede Bayesiana se comporta nisso:

In [28]:
predictions = bn.predict(discretised_data, "G1")


In [29]:
print(f"The prediction is '{predictions.loc[18, 'G1_prediction']}'")


The prediction is 'Fail'


A previsão da Rede Bayesiana é que o aluno reprovou. Vamos comparar isso com o dado real:

In [30]:
print(f"The ground truth is '{discretised_data.loc[18, 'G1']}'")


The ground truth is 'Fail'


#### Avaliando nosso Modelo

In [31]:
classification_report(bn, test, "G1")


{'G1_Fail': {'precision': 0.7777777777777778,
  'recall': 0.5833333333333334,
  'f1-score': 0.6666666666666666,
  'support': 12.0},
 'G1_Pass': {'precision': 0.9107142857142857,
  'recall': 0.9622641509433962,
  'f1-score': 0.9357798165137615,
  'support': 53.0},
 'accuracy': 0.8923076923076924,
 'macro avg': {'precision': 0.8442460317460317,
  'recall': 0.7727987421383649,
  'f1-score': 0.8012232415902141,
  'support': 65.0},
 'weighted avg': {'precision': 0.8861721611721611,
  'recall': 0.8923076923076924,
  'f1-score': 0.8860973888496825,
  'support': 65.0}}

Este relatório mostra que o modelo que definimos é capaz de classificar se um aluno passa razoavelmente bem no exame.

Para as predições em que o aluno reprova, a precision é boa, mas a recall é ruim. Isso implica que podemos confiar nas previsões para essa classe quando elas são feitas, mas é provável que percamos algumas das previsões que deveríamos ter feito. Talvez essas previsões erroneas sejam resultado de algo ausente em nossa estrutura - essa pode ser uma área interessante a ser explorada.

In [32]:
roc, auc = roc_auc(bn, test, "G1")
print(auc)

0.9181065088757396


O valor AUC do nosso modelo é alto, o que nos dá confiança no desempenho.

#### Querying Marginals

Depois de iterar sobre nossa estrutura de modelo, CPDs e validar a qualidade de nosso modelo, podemos consultar nosso modelo sob diferentes observações para obter insights.

##### Baseline Marginals

Para consultar o modelo para nosso Baseline Marginals que refletem a população como um todo, um método de consulta pode ser usado. Primeiro, vamos atualizar nosso modelo usando o conjunto de dados completo, pois o que temos atualmente foi construído apenas a partir de dados de treinamento.

In [33]:
bn = bn.fit_cpds(discretised_data, method="BayesianEstimator", bayes_prior="K2")


Para inferência, devemos criar uma nova InferenceEngine da nossa BayesianNetwork, que nos permite consultar o modelo. O método de consulta calculará a probabilidade marginal de todos os estados para todos os nós.

In [34]:
ie = InferenceEngine(bn)
marginals = ie.query()
marginals["G1"]

{'Fail': 0.25260687281677224, 'Pass': 0.7473931271832277}

A saída observada nos diz que P(G1=Fail) = 0,25 e P(G1=Pass) = 0,75. Como uma rápida teste de sanidade, podemos calcular qual proporção de nosso conjunto de dados é 'fail', que deve ser aproximadamente a mesma.

In [35]:
labels, counts = np.unique(discretised_data["G1"], return_counts=True)
list(zip(labels, counts))

157/157+492

[('Fail', 157), ('Pass', 492)]

Marginais após Observações (Contrafactuais)

Também podemos consultar a probabilidade marginal dos estados em nossa rede, dadas algumas observações. Essas observações podem ser feitas em qualquer ponto da rede e seu impacto será propagado até o nó de interesse.

Vejamos a diferença na probabilidade de G1 com base no tempo de estudo.

In [36]:
marginals_short = ie.query({"studytime": "short-studytime"})

marginals_long = ie.query({"studytime": "long-studytime"})

print("Marginal G1 | Short Studtyime", marginals_short["G1"])
print("Marginal G1 | Long Studytime", marginals_long["G1"])

Marginal G1 | Short Studtyime {'Fail': 0.2776556433482524, 'Pass': 0.7223443566517477}
Marginal G1 | Long Studytime {'Fail': 0.15504850337837614, 'Pass': 0.8449514966216239}


#### Intervenções (Do-Calculus)

O CausalNex também oferece suporte ao Do-Calculus simples, permitindo-nos especificar intervenções.

##### Atualizando uma distribuição

Podemos aplicar uma intervenção a qualquer nó em nossos dados, atualizando sua distribuição usando um operador *do*. Isso pode ser pensado como perguntar ao nosso modelo “E se” algo fosse diferente. Por exemplo, poderíamos perguntar o que aconteceria se 100% dos alunos quisessem fazer o ensino superior.

In [37]:

print("distribution before do", ie.query()["higher"])

ie.do_intervention("higher",
                   {'yes': 1.0,
                    'no': 0.0})

print("distribution after do", ie.query()["higher"])

distribution before do {'no': 0.10752688172043011, 'yes': 0.8924731182795698}
distribution after do {'no': 0.0, 'yes': 0.9999999999999998}


Podemos redefinir quaisquer intervenções que fizermos usando o método reset_intervention e fornecendo o nó que queremos redefinir.

In [38]:
ie.reset_do("higher")

##### Efeito da intervenção nas probabilidades

Podemos novamente usar a consulta para examinar o efeito que uma intervenção tem em nossas probabilidades marginais. Nesse caso, podemos observar como a probabilidade de aprovação muda se 100% dos alunos quiserem fazer o ensino superior.

In [39]:
print("marginal G1", ie.query()["G1"])

ie.do_intervention("higher",
                   {'yes': 1.0,
                    'no': 0.0})

print("updated marginal G1", ie.query()["G1"])

marginal G1 {'Fail': 0.25260687281677224, 'Pass': 0.7473931271832277}
updated marginal G1 {'Fail': 0.20682952942551894, 'Pass': 0.7931704705744809}


Nesse caso, podemos ver que, se 100% dos alunos quisessem fazer ensino superior (em oposição a 90% em nossa população de dados), estimamos que a taxa de aprovação aumentaria de 74,7% para 79,3%.